In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
def get_data_loader(training = True):
#     transform=transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.1307,), (0.3081,))
#         ])
    custom_transform= transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    train_set = datasets.MNIST('./data', train=True, download=True,
                       transform=custom_transform)
    test_set = datasets.MNIST('./data', train=False,download=True,
                       transform=custom_transform)
    
    if training == True:
        return torch.utils.data.DataLoader(train_set, batch_size = 50)
    else:
        return torch.utils.data.DataLoader(test_set, batch_size = 50)
    
get_data_loader()

In [4]:
train_loader = get_data_loader()
print(type(train_loader))
print(train_loader.dataset)
test_loader = get_data_loader(False)

<class 'torch.utils.data.dataloader.DataLoader'>
Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )


In [5]:
def build_model():
    model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28,128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 10),
    )
    return model 

model = build_model()
print(model)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=128, bias=True)
  (2): ReLU()
  (3): Linear(in_features=128, out_features=64, bias=True)
  (4): ReLU()
  (5): Linear(in_features=64, out_features=10, bias=True)
)


In [6]:
criterion = nn.CrossEntropyLoss()
def train_model(model, train_loader, criterion, T):
    opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    model.train()
    for epochs in range(T):
        correct = 0
        total = 0
        for images,labels in train_loader:
            opt.zero_grad()
            output=model(images)
            loss=criterion(output,labels)
            predicted=output.argmax(dim=1,keepdim=True)
            total += labels.size(0)
            correct += predicted.eq(labels.view_as(predicted)).sum().item()
            loss.backward()
            opt.step()
        print('Train Epoch:{} Accuracy:{}/{}({:.2f}%) Loss:{:.3f}'.format(epochs, correct,total,100*(correct/total),loss.item()))
    
train_model(model, train_loader, criterion, T = 5)

Train Epoch:0 Accuracy:47903/60000(79.84%) Loss:0.233
Train Epoch:1 Accuracy:54845/60000(91.41%) Loss:0.140
Train Epoch:2 Accuracy:55868/60000(93.11%) Loss:0.090
Train Epoch:3 Accuracy:56580/60000(94.30%) Loss:0.062
Train Epoch:4 Accuracy:57102/60000(95.17%) Loss:0.047


In [11]:
def evaluate_model(model, test_loader, criterion, show_loss = True):
    opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    model.eval()
    total = 0
    correct=0
    running_loss=0
    with torch.no_grad():
        for data, labels in test_loader:
            output=model(data)
            running_loss+=criterion(output,labels).item()
            predicted=output.argmax(dim=1,keepdim=True)
            total += labels.size(0)
            correct += predicted.eq(labels.view_as(predicted)).sum().item()
    running_loss=running_loss/len(test_loader)
    if show_loss == True:
        print('Average loss:{:.4f}\nAccuracy:{:.2f}%'.format(running_loss,100*(correct/total)))
    else: 
        print('Accuracy:{:.2f}%'.format(100*(correct/total)))
            
evaluate_model(model, test_loader, criterion, show_loss = False)
evaluate_model(model, test_loader, criterion, show_loss = True)

Average loss:0.1619
Accuracy:95.19%


In [13]:
image=[] 
l=[]
for data, labels in test_loader:
        image.append(data)
        l.append(labels)
image

[tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           ...,
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],
 
 
         [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           ...,
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],
 
 
         [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
       

In [26]:
def predict_label(model, test_images, index):
    model.eval()
    output=model(test_images)
    prob = F.softmax(output, dim=1)
#     print(prob)
    class_names = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine']
    d={}
    for i in range(len(class_names)):
        d[class_names[i]]=float(prob[index][i])*100
    d={k: v for k, v in sorted(d.items(), key=lambda item: item[1],reverse=True)}
    for i in list(d.items())[:3]:
        print('{}:{:.2f}%'.format(i[0],i[1]))
    
predict_label(model, image[0], 1)

two:98.85%
three:0.86%
one:0.10%
